# 知識點
- Inception v1 發表於 2014 年，其重要點為：
    1. 引進 Inception 層的概念：目的在於使用不同大小的 Kernels 以結合不同 Receptive Field 的特徵圖，獲得不同尺度的訊息。
        <img src="https://www.programmersought.com/images/737/c115b23e3d8e8431590a21542105dfb1.png" width=500>
        
        因為信息位置的巨大差異，所以不同大小的卷積核有不同功能：
        - 信息分佈更全局性的圖像偏好較大的卷積核
        - 信息分佈比較局部的圖像偏好較小的卷積核
        
       為了確保輸出的特徵圖在長寬上擁有一樣尺寸，就得使用 Padding 技巧，1 x 1 kernel 輸出大小與輸入相同
       而 3 x 3、5 x 5 kernel 則分別設定補邊值為 1、2，在 tensorflow、Keras 中最快的方式就是設定 `padding='same'`，就能在步長為1時確保輸出尺寸維持相同。



    2. 使用 1 x 1 卷積層做降維：經過 1 x 1 kernel 的壓縮，可以控制輸出 Channels 的深度，並同時能增加模型的非線性。
       它的好處在於能用相當少的參數量，達到壓縮特徵圖深度的目的。
       舉例來說，當輸入 Feature Map 為 (batch_size, 14, 14, 192)，要將其壓縮為 (batch_size, 14, 14, n)
       1 x 1 kernel 只需要 1x1x192xn+n 個參數量，當然同樣事情也可以用 3 x 3 kernel 達成，但參數量就會變為 3x3x192xn+n。
    
    另外 Inception v1 還使用了輔助分類器 (Auxiliary classifiers)，這個概念在之後的文獻中比上少看見，原論文中提到其存在的目的在於通過中間層的分類損失，監督神經網路，最後會按一個較小的權重（0.3）加到最終分類結果中，這樣相當於做了模型融合，同時給網絡增加了反向傳播的梯度信號，也提供了額外的正則化。
<img src="https://cdn-images-1.medium.com/max/1000/1*63ZlzSqS851I3nW89PUVlg.jpeg">


    
- Inception v2 的重點則在引入了 Batch Normalization。

- Inception v3 的重點如下：
    - 將 n x n 的卷積核以 1 x n + n x 1 的卷積核取代。 
    - 將 1 個卷積拆成 2 個卷積，使得網絡深度更深，增加了網絡的非線性（每增加一層都要經過Activation Function）。<br>
        如原本 3 x 3 的 Kernel 就會被拆解為 1x3 + 3x1 使得 Inception Block 架構變得更深。<br>
        然而原文也提及，此種結構放在神經網路前幾層效果並不好，建議放到較深的層數中使用。

## 延伸閱讀
[Network In Network](https://arxiv.org/abs/1312.4400) — 首次提出 1 x 1 convolution降維。


## 『本次練習內容』
#### 學習如何搭建Inception Block

## 『本次練習目的』
  #### 了解Inception原理
  #### 了解如何導入Inception block到原本架構中

---

![Incpeiton](img/Inception架構.png)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K

## 導入InceptionV2-有BatchNormalization的Convolution

In [2]:
def Conv2d_bn(x, filters, kernel_size, padding='same', strides=(1, 1),
              normalizer=True, activation='relu', name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    
    x = layers.Conv2D(filters, kernel_size, 
                      strides=strides, padding=padding, 
                      use_bias=False, name=conv_name)(x)
    if normalizer:
        x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = layers.Activation(activation, name=act_name)(x)
    return x

## 參考上圖搭建 InceptionV1_block

In [3]:
def InceptionV1_block(x, specs, channel_axis, name):
    (br0, br1, br2, br3) = specs # ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+'_Branch_0')
    
    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 3), name=name+"_Branch_1_1")
    
    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2")
    branch_2 = Conv2d_bn(branch_2, br2[1], (5, 5), name=name+"_Branch_2_1")
    
    branch_3 = layers.MaxPool2D((3, 3), 1, padding='same', name=name+"_Branch_3")(x)
    branch_3 = Conv2d_bn(branch_3, br3[0], (1, 1), name=name+"_Branch_3_1")
    
    x = layers.concatenate([branch_0, branch_1, branch_2, branch_3],
                           axis=channel_axis, 
                           name=name+"_Concatenated")
    return x

## 測試

In [4]:
img_input = layers.Input(shape=(224, 224, 1))

x = InceptionV1_block(img_input, ((64, ), (96, 128), (16, 32), (32, )), 3, 'Block_1')

print(x)

Tensor("Block_1_Concatenated/Identity:0", shape=(None, 224, 224, 256), dtype=float32)


## 將 InceptionV1_block中n*n卷積改為1 x n+n x 1

In [5]:
def InceptionV3_block(x, specs, channels_axis, name):
    (br0, br1, br2, br3) = specs # ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0")
    
    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (1, 3), name=name+"_Branch_1_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 1), name=name+"_Branch_1_2")
    
    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2")
    branch_2 = Conv2d_bn(branch_2, br2[1], (1, 5), name=name+"_Branch_2_1")
    branch_2 = Conv2d_bn(branch_2, br2[1], (5, 1), name=name+"_Branch_2_2")
    
    branch_3 = layers.MaxPool2D((3, 3), 1, padding='same', name=name+"_Branch_3")(x)
    branch_3 = Conv2d_bn(branch_3, br3[0], (1, 1), name=name+"_Branch_3_1")
    
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channels_axis,
        name=name+"_Concatenated")
    return x

## 測試

In [6]:
img_input = layers.Input(shape=(224,224,1))
x=InceptionV3_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
print(x)

Tensor("Block_1_Concatenated_1/Identity:0", shape=(None, 224, 224, 256), dtype=float32)


---

## 額外練習

## 將VGG16 Block_3中的Convolution全部改為InceptionV1_block
## Block_5中的Convolution全部改為InceptionV3_block
## 並將所有Convolution改為Conv2d_bn

#### 原vgg16架構

In [7]:
def VGG16(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

   
    return model

#### 修改後

In [8]:
def VGG16_Inception(include_top=True, input_tensor=None, input_shape=(224, 224, 1),
          pooling='max', classes=1000):
    
    image_input = layers.Input(shape=input_shape)
    
    x = Conv2d_bn(image_input, 64, (3, 3), name="Block1_conv1")
    x = Conv2d_bn(x, 64, (3, 3), name="Block1_conv2")
    x = layers.MaxPool2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    x = Conv2d_bn(x, 128, (3, 3), name="Block2_conv1")
    x = Conv2d_bn(x, 128, (3, 3), name="Block2_conv2")
    x = layers.MaxPool2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    x = InceptionV1_block(x, ((64, ), (96, 128), (16, 32), (32, )), 3, 'Block_1')
    x = InceptionV1_block(x, ((64, ), (96, 128), (16, 32), (32, )), 3, 'Block_2')
    x = InceptionV1_block(x, ((64, ), (96, 128), (16, 32), (32, )), 3, 'Block_3')
    x = layers.MaxPool2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
    x = Conv2d_bn(x, 512, (3, 3), name="Block4_conv1")
    x = Conv2d_bn(x, 512, (3, 3), name="Block4_conv2")
    x = Conv2d_bn(x, 512, (3, 3), name="Block4_conv3")
    x = layers.MaxPool2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    
    x = InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_4')
    x = InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_5')
    x = InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_6')
    x = layers.MaxPool2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D()(x)

    inputs = image_input
    # Create model.
    model = models.Model(inputs, x, name='vgg16')
    
    return model

In [9]:
model = VGG16_Inception(include_top=False)
model.summary()

Model: "vgg16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
Block1_conv1_conv (Conv2D)      (None, 224, 224, 64) 576         input_3[0][0]                    
__________________________________________________________________________________________________
Block1_conv1_bn (BatchNormaliza (None, 224, 224, 64) 192         Block1_conv1_conv[0][0]          
__________________________________________________________________________________________________
Block1_conv1_act (Activation)   (None, 224, 224, 64) 0           Block1_conv1_bn[0][0]            
______________________________________________________________________________________________